In [1]:
import nidaqmx
from nidaqmx.constants import AcquisitionType, Edge
import time

## System

In [7]:
system = nidaqmx.system.System.local()

In [9]:
for dev in system.devices:
    print(dev)

Device(name=PXI-6363)
Device(name=PXI-6723)
Device(name=PXI-6733)


## Basics 

### task.read()

In [46]:
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("PXI-6363/ai10")
    res = task.read(number_of_samples_per_channel=5)
    print(res)

[-0.49668359976606874, -0.4979718989327781, -0.49813293632877015, -0.49813293632877015, -0.498616048516951]


In [72]:
with nidaqmx.Task() as task:
    task.ai_channels.add_ai_voltage_chan("PXI-6363/ai2")
    task.timing.cfg_samp_clk_timing(rate=10, samps_per_chan=5)
    res = task.read(number_of_samples_per_channel=5)
    print(res)

[5.278183554395748, 5.278183554395748, 5.278183554395748, 5.278183554395748, 5.278183554395748]


In [49]:
from nidaqmx.constants import LineGrouping

In [52]:
with nidaqmx.Task() as task:
    task.di_channels.add_di_chan('PXI-6363/port0/line0:7', line_grouping=LineGrouping.CHAN_PER_LINE)
    print(task.read())

[False, False, False, False, False, False, False, False]


### task.write()

In [4]:
with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan('PXI-6363/ao0')
    task.write(0, auto_start=True)

In [53]:
from nidaqmx.types import CtrTime

In [59]:
with nidaqmx.Task() as task:
    task.co_channels.add_co_pulse_chan_time('PXI-6363/ctr0')
    sample = CtrTime(high_time=0.1, low_time=0.1)
    task.write(sample)

DaqError: Specified operation did not complete, because the specified timeout expired.
Task Name: _unnamedTask<2C>

Status Code: -200474

In [77]:
with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan('PXI-6363/ao0')
    
    task.timing.cfg_samp_clk_timing(rate=1, samps_per_chan=5)
    task.triggers
    
    task.write([0,1,2,3,4], auto_start=True)

## Timing

### One task

In [199]:
with nidaqmx.Task() as task: 
    task.ao_channels.add_ao_voltage_chan('PXI-6363/ao0')
    
    task.timing.cfg_samp_clk_timing(rate=20, sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=4)
    
    print(task.write(data=[0,1,2]))
    task.start()
    time.sleep(5)
    task.stop()

3


In [188]:
with nidaqmx.Task() as task: 
    task.ao_channels.add_ao_voltage_chan('PXI-6363/ao0')
    
    task.timing.cfg_samp_clk_timing(rate=2, sample_mode=AcquisitionType.FINITE, samps_per_chan=11)
    
    print(task.write(data=[0,1,2,3,4,5,6,7,1,2,0], auto_start=True))
    task.wait_until_done()
    task.stop()

11


In [194]:
with nidaqmx.Task() as task: 
    task.ao_channels.add_ao_voltage_chan('PXI-6363/ao0')
    
    task.timing.ref_clk_src = "PXIe_Clk100"
    task.timing.cfg_samp_clk_timing(
        rate=20,
        sample_mode=AcquisitionType.CONTINUOUS,
        samps_per_chan=4
    )
    
    print(task.write(data=[0,1,2], auto_start=True))
    time.sleep(5)
    task.stop()

3


#### External SampleClock source

External clock source: digital output

In [ ]:
with nidaqmx.Task() as task: 
    task.do_channels.add_do_chan('/PXI-6363/port0/line8')
    
    task.timing.cfg_samp_clk_timing(
        rate=1,
        sample_mode=AcquisitionType.CONTINUOUS,
        samps_per_chan=2
    )
    
    task.write([False, True])
    task.start()
    time.sleep(100)
    task.stop()

##### Deliver the SampleClock signal by a wire from '/PXI-6363/port0/line8' to '/PXI-6363/PFI12'

DI port will be used to input the external clock signal
Test that the logical signal arrives

In [286]:
with nidaqmx.Task() as task: 
    task.di_channels.add_di_chan('/PXI-6363/PFI12')
    
    print(task.read())

False


Generate AO signal using external clock from '/PXI-6363/PFI12'

In [291]:
with nidaqmx.Task() as task: 
    task.ao_channels.add_ao_voltage_chan('PXI-6363/ao0')
    
    task.timing.cfg_samp_clk_timing(
        rate=10,
        source = '/PXI-6363/PFI12',
        active_edge = Edge.RISING,
        sample_mode=AcquisitionType.FINITE,
        samps_per_chan=9
    )
    
    print(task.write(data=[0,1,2,3,4,5,1,2,0]))
    task.start()
    task.wait_until_done(timeout=100)
    task.stop()

9


##### Deliver the SampleClock signal by internal connection from '/PXI-6363/port0/line8' to '/PXI-6363/PFI12'

In [307]:
with nidaqmx.Task() as task: 
    task.ao_channels.add_ao_voltage_chan('PXI-6363/ao0')
    
    task.timing.cfg_samp_clk_timing(
        rate=10,
        source = '/PXI-6363/do/SampleClock',
        active_edge = Edge.RISING,
        sample_mode=AcquisitionType.CONTINUOUS,
        samps_per_chan=9
    )
    
    print(task.write(data=[0,1,2,3,4,5,1,2,0]))
    task.start()
    task.wait_until_done(timeout=100)
    task.stop()

9


DaqError: Wait Until Done did not indicate that the task was done within the specified timeout.

Increase the timeout, check the program, and make sure connections for external timing and triggering are in place.
Task Name: _unnamedTask<132>

Status Code: -200560

### Two tasks

In [211]:
with nidaqmx.Task() as task0, nidaqmx.Task() as task1:
    
    task0.ao_channels.add_ao_voltage_chan('PXI-6363/ao0')
    task1.ai_channels.add_ai_voltage_chan('PXI-6363/ai0')
    
    task0.timing.cfg_samp_clk_timing(
        rate=20,
        sample_mode=AcquisitionType.CONTINUOUS,
        samps_per_chan=4
    )
    
    task1.timing.cfg_samp_clk_timing(
        rate=20,
        source = '/PXI-6363/ao/SampleClock',
        sample_mode=AcquisitionType.CONTINUOUS,
        samps_per_chan=4
    )
    
    print(task0.timing.samp_clk_src)
    print(task1.timing.samp_clk_src)
    
    task1.start()
    
    task0.write([0,1,2,3])
    task0.start()
    
    time.sleep(5)
    task0.stop()
    task1.stop()
#     data = task1.read()
    
    

/PXI-6363/ao/SampleClockTimebase
/PXI-6363/ao/SampleClock


In [ ]:
with nidaqmx.Task() as task:
    task.ao_channels.add_ao_voltage_chan('/PXI-')